In [12]:
!conda install pytorch torchvision torchaudio pytorch-cuda=11.7 -c pytorch -c nvidia -y
!conda install -c conda-forge tensorboard

Solving environment: - ^C
failed with initial frozen solve. Retrying with flexible solve.

CondaError: KeyboardInterrupt

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 22.11.1

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Retrieving notices: ...working... done


In [13]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | bash
!apt install -y git-lfs -y
!git lfs install

Detected operating system as debian/10.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Detected apt version as 1.8.2.1
Running apt-get update... done.
Installing debian-archive-keyring which is needed for installing 
apt-transport-https on many Debian systems.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... Packagecloud gpg key imported to /etc/apt/keyrings/github_git-lfs-archive-keyring.gpg
done.
Running apt-get update... done.

The repository is setup! You can now install packages.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (3.3.0).
0 upgraded, 0 newly installed, 0 to remove and 55 not upgraded.
Git LFS initialized.


In [14]:
import torch
torch.cuda.get_device_name(0)


'Tesla T4'

In [15]:
!add-apt-repository -y ppa:jonathonf/ffmpeg-4
!apt update
!apt install -y ffmpeg


/bin/bash: add-apt-repository: command not found
Hit:1 http://security.debian.org/debian-security buster/updates InRelease
Hit:2 http://deb.debian.org/debian buster InRelease                     
Hit:3 http://deb.debian.org/debian buster-updates InRelease             
Hit:4 https://packagecloud.io/github/git-lfs/debian buster InRelease    
Reading package lists... Done
Building dependency tree       
Reading state information... Done
55 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:4.1.10-0+deb10u1).
0 upgraded, 0 newly installed, 0 to remove and 55 not upgraded.


In [16]:
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.3.0
!pip install jiwer
!pip install gradio


Looking in indexes: https://pypi.org/simple, https://packagecloud.io/github/git-lfs/pypi/simple
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-a8yel72h
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-a8yel72h
  Resolved https://github.com/huggingface/transformers to commit 35a7052b61579cfe8df1a059d4cd3359310ec2d1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://pypi.org/simple, https://packagecloud.io/github/git-lfs/pypi/simple
Looking in indexes: https://pypi.org/simple, https://packagecloud.io/github/git-lfs/pypi/simple
Looking in indexes: https://pypi.org/simple, https://packagecloud.io/github/git-lfs/pypi/simple


In [23]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

train_dataset, test_dataset, validation_dataset = load_dataset("mozilla-foundation/common_voice_11_0", "zh-CN", split=["train", "validation", "test"], use_auth_token=True)


Found cached dataset common_voice_11_0 (/root/.cache/huggingface/datasets/mozilla-foundation___common_voice_11_0/zh-CN/11.0.0/f8e47235d9b4e68fa24ed71d63266a02018ccf7194b2a8c9c598a5f3ab304d9f)


  0%|          | 0/3 [00:00<?, ?it/s]

In [24]:
common_voice["train"]=train_dataset
common_voice["validation"]=validation_dataset
common_voice["test"]=test_dataset
print(common_voice)

common_voice.save_to_disk('zh-common-voice-original')

DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 29056
    })
    validation: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 10581
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 10581
    })
})


Saving the dataset (0/3 shards):   0%|          | 0/29056 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10581 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10581 [00:00<?, ? examples/s]

In [25]:
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])


In [26]:
common_voice

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 29056
    })
    validation: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 10581
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 10581
    })
})

In [27]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Chinese", task="transcribe")
tokenizer = processor.tokenizer

In [28]:
input_str = common_voice["train"][0]["sentence"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")


Input:                 性喜温暖润湿气候且耐寒。
Decoded w/ special:    <|startoftranscript|><|zh|><|transcribe|><|notimestamps|>性喜温暖润湿气候且耐寒。<|endoftext|>
Decoded w/out special: 性喜温暖润湿气候且耐寒。
Are equal:             True


In [30]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))
print(common_voice["train"][0])


{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/892af1724096c1e9ed1d53df5f8c14697ce16fc1e756f4c7b56c93aa20bc0625/common_voice_zh-CN_33211332.mp3', 'array': array([-9.3524797e-13, -1.4627063e-12,  1.9229006e-12, ...,
       -5.5482551e-06,  2.6667838e-05,  1.3390082e-05], dtype=float32), 'sampling_rate': 16000}, 'sentence': '性喜温暖润湿气候且耐寒。'}


In [32]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array 
    batch["input_features"] = processor.feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids 
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch


In [33]:
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=4)


#0:   0%|          | 0/7264 [00:00<?, ?ex/s]

#1:   0%|          | 0/7264 [00:00<?, ?ex/s]

#2:   0%|          | 0/7264 [00:00<?, ?ex/s]

#3:   0%|          | 0/7264 [00:00<?, ?ex/s]

#0:   0%|          | 0/2646 [00:00<?, ?ex/s]

...

#3:   0%|          | 0/2645 [00:00<?, ?ex/s]

#2:   0%|          | 0/2645 [00:00<?, ?ex/s]

#1:   0%|          | 0/2645 [00:00<?, ?ex/s]

#0:   0%|          | 0/2646 [00:00<?, ?ex/s]

#3:   0%|          | 0/2645 [00:00<?, ?ex/s]

In [34]:
common_voice.save_to_disk("zhcn-common-voice-processed")

Saving the dataset (0/56 shards):   0%|          | 0/29056 [00:00<?, ? examples/s]

Saving the dataset (0/21 shards):   0%|          | 0/10581 [00:00<?, ? examples/s]

Saving the dataset (0/21 shards):   0%|          | 0/10581 [00:00<?, ? examples/s]

In [35]:
import boto3
import sagemaker
import os
from sagemaker import get_execution_role

sess = sagemaker.Session()
ROLE = get_execution_role()

BUCKET = sess.default_bucket() 
BUCKET
PREFIX = "whisper/data/zhcn-common-voice-processed"

In [36]:
!aws s3 cp --recursive zhtw-common-voice-processed s3://sagemaker-us-east-1-348052051973/whisper/data/zhcn-common-voice-processed

upload: zhtw-common-voice-processed/dataset_dict.json to s3://sagemaker-us-east-1-348052051973/whisper/data/zhcn-common-voice-processed/dataset_dict.json
upload: zhtw-common-voice-processed/test/data-00000-of-00010.arrow to s3://sagemaker-us-east-1-348052051973/whisper/data/zhcn-common-voice-processed/test/data-00000-of-00010.arrow
upload: zhtw-common-voice-processed/test/data-00001-of-00010.arrow to s3://sagemaker-us-east-1-348052051973/whisper/data/zhcn-common-voice-processed/test/data-00001-of-00010.arrow
upload: zhtw-common-voice-processed/test/data-00003-of-00010.arrow to s3://sagemaker-us-east-1-348052051973/whisper/data/zhcn-common-voice-processed/test/data-00003-of-00010.arrow
upload: zhtw-common-voice-processed/test/data-00002-of-00010.arrow to s3://sagemaker-us-east-1-348052051973/whisper/data/zhcn-common-voice-processed/test/data-00002-of-00010.arrow
upload: zhtw-common-voice-processed/test/data-00004-of-00010.arrow to s3://sagemaker-us-east-1-348052051973/whisper/data/zhcn-